# Importance Sampling

The $J/\Psi \rightarrow \gamma \pi^0 \pi^0$ decay is used here (which has a narrow $\omega$ state)

Let's go!

## Step 1: Define intensity
First we create the intensity. If this step looks unfamiliar to you, you might want to check other intensity construction examples using the expert system.

In [ ]:
from pycompwa.expertsystem.ui.system_control import (
    StateTransitionManager, InteractionTypes)
from pycompwa.expertsystem.amplitude.helicitydecay import (
    HelicityAmplitudeGeneratorXML)
from pycompwa.expertsystem.topology.graph import (
    get_intermediate_state_edges)
from pycompwa.expertsystem.state.particle import (
    particle_list)
    
initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma", [-1, 1]), ("pi0", [0]), ("pi0", [0])]

tbd_manager = StateTransitionManager(initial_state, final_state,
                                     formalism_type='helicity',
                                     topology_building='isobar')

# The omega is so narrow that the hit&miss generation takes way too long.
# Therefore we increase the width artificially in this example!
omega = particle_list["omega(782)"]
parameters = omega['DecayInfo']['Parameter']
for par in parameters:
    if par['@Type'] == 'Width':
        par['Value'] = 0.001

tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong, InteractionTypes.EM])
tbd_manager.allowed_intermediate_particles = ['f2(1270)', 'omega']
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")

xml_generator = HelicityAmplitudeGeneratorXML()
xml_generator.generate(solutions)
xml_generator.write_to_file('model.xml')

## Step 2: Create intensity and generate importance weighted phase space sample
Now we generate a phase space sample which is importance sampled by the intensity. This can easily be achieved by using the `generate_importance_sampled_phsp()` generate helper function.

In [ ]:
# pycompwa is the python interface to ComPWA's c++ modules
import pycompwa.ui as pwa

particles = pwa.read_particles('model.xml')

kin = pwa.create_helicity_kinematics('model.xml', particles)

# generate phase space sample (flat) used for amplitude normalization
gen = pwa.EvtGenGenerator(kin.get_particle_state_transition_kinematics_info())

rand_gen = pwa.StdUniformRealGenerator(12345)

phsp_sample = pwa.generate_phsp(100000, gen, rand_gen)

# create intensity
builder = pwa.IntensityBuilderXML('model.xml', particles, kin, phsp_sample)
intensity = builder.create_intensity()

# generate importance sampled phase space sample
phsp_sample_importance = pwa.generate_importance_sampled_phsp(2000, kin, gen, intensity, rand_gen)

## Step 3: Visualize the phase space sample
This phase space sample can be visualized in a Dalitz plot. There one expects more events, where the intensity is large, but still an overall uniform distribution. First create all subsystems. Since in this example, we have both $f$ and $\omega$ resonances, all subsystems already exist and this step is redundant.

In [ ]:
kin.create_all_subsystems()
data_set = kin.convert(phsp_sample_importance)

Then we can use `pandas` and `matplotlib` to create a Dalitz plot:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
id_to_name = kin.get_particle_state_transition_kinematics_info().get_final_state_id_to_name_mapping()
mass_x = 'mSq_(3,4)'
mass_y = 'mSq_(2,4)'
plt.hist2d(
    data_set.data[mass_x],
    data_set.data[mass_y],
    bins=50,
);
for id, name in id_to_name.items():
    mass_x = mass_x.replace(str(id), name)
    mass_y = mass_x.replace(str(id), name)
plt.xlabel(mass_x);
plt.ylabel(mass_y);